# Name: Hanan Fared Mohamed Omara , ID =  20398559

- To ensure academic integrity, I utilized some code snippets from tutorial_2 part2 which available in the OnQ.By using these code snoppets as a reference ans building upon them, I was able to better understand and implement the required tasks in my project.

In [36]:
# Import libraries
# i am using python 3.10.11 for this tutorial. We suggest you first install bertopic library and then add other missing libraries to avoid library compatability issues.
import pandas as pd                                        
import numpy as np                                        
import matplotlib.pyplot as plt                           
import seaborn as sns  
!pip install bertopic
from bertopic import BERTopic
from tqdm import tqdm

# the following code ensures that you can see your (print) results for multiple tasks within a coding block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.options.display.width = None

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz (538.1 MB)
  Preparing metadata (setup.py) ... done


In [38]:
!pip install spacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS 
from IPython.utils import io
# with io.capture_output() as captured:
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz
# !pip install en_core_sci_lg
import en_core_sci_lg  # import downlaoded model
import string
!pip install minisom
from minisom import MiniSom  
from sklearn.cluster import SpectralClustering 
import scipy.cluster.hierarchy as sch
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Lab 1 contains 4 analysis questions. 
The dataset is reused from tutorial part 2, i.e., the sampled_arxiv_cs.csv file.

Question 1: Load in the provided dataset and summarize the basic statistics. Specifically your code should answer:

1) What are the feature types and their basic statistics (using describe(include='all'))
2) How many features have missing values? 

In [39]:
# put your code for Q1 here, you can have multiple code blocks.
# as usual, let's first check the basic statistics of the dataset, it should contain 100k recordds.
df = pd.read_csv('sampled_arxiv_cs.csv')
df.head()
df.info()
df.describe(include="all")

<ipython-input-39-8f102609e9cf>:3: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.



,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,2006.15607,Youngwan Lee,"Youngwan Lee, Joong-won Hwang, Hyung-Il Kim, K...",Localization Uncertainty Estimation for Anchor...,NaN,NaN,NaN,NaN,cs.CV,http://arxiv.org/licenses/nonexclusive-distrib...,"Since many safety-critical systems, such as ...","[{'version': 'v1', 'created': 'Sun, 28 Jun 202...",2022-07-07,"[['Lee', 'Youngwan', ''], ['Hwang', 'Joong-won..."
1,1507.03782,Helmut Strobel,"Helmut Strobel, Wolfgang Muessel, Daniel Linne...",Fisher Information and entanglement of non-Gau...,"5 pages, 4 figures, 7 pages supplementary mate...","Science 345, 424-427 (2014)",10.1126/science.1250147,NaN,quant-ph cond-mat.quant-gas physics.atom-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Entanglement is the key quantum resource for...,"[{'version': 'v1', 'created': 'Tue, 14 Jul 201...",2015-07-15,"[['Strobel', 'Helmut', ''], ['Muessel', 'Wolfg..."
2,901.4201,Stephane Martin,"Denis Lugiez (LIF), St\'ephane Martin (LIF)",Peer to Peer Optimistic Collaborative Editing ...,NaN,NaN,NaN,NaN,cs.DS,http://arxiv.org/licenses/nonexclusive-distrib...,Collaborative editing consists in editing a ...,"[{'version': 'v1', 'created': 'Tue, 27 Jan 200...",2009-01-28,"[['Lugiez', 'Denis', '', 'LIF'], ['Martin', 'S..."
3,1802.05814,Dawen Liang,"Dawen Liang, Rahul G. Krishnan, Matthew D. Hof...",Variational Autoencoders for Collaborative Fil...,"10 pages, 3 figures. WWW 2018",NaN,NaN,NaN,stat.ML cs.IR cs.LG,http://arxiv.org/licenses/nonexclusive-distrib...,We extend variational autoencoders (VAEs) to...,"[{'version': 'v1', 'created': 'Fri, 16 Feb 201...",2018-02-19,"[['Liang', 'Dawen', ''], ['Krishnan', 'Rahul G..."
4,1010.5069,Philippe Maincon,Philippe Maincon,A Wiener-Laguerre model of VIV forces given re...,NaN,NaN,NaN,NaN,physics.flu-dyn,http://arxiv.org/licenses/nonexclusive-distrib...,Slender structures immersed in a cross flow ...,"[{'version': 'v1', 'created': 'Mon, 25 Oct 201...",2010-10-26,"[['Maincon', 'Philippe', '']]"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              50000 non-null  object
 1   submitter       49959 non-null  object
 2   authors         50000 non-null  object
 3   title           50000 non-null  object
 4   comments        31780 non-null  object
 5   journal-ref     10783 non-null  object
 6   doi             15365 non-null  object
 7   report-no       1170 non-null   object
 8   categories      50000 non-null  object
 9   license         47468 non-null  object
 10  abstract        50000 non-null  object
 11  versions        50000 non-null  object
 12  update_date     50000 non-null  object
 13  authors_parsed  50000 non-null  object
dtypes: object(14)
memory usage: 10.7+ MB


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
count,50000,49959,50000,50000,31780,10783,15365,1170,50000,47468,50000,50000,50000,50000
unique,50000,40229,48953,49998,23292,10709,15362,1157,8908,9,49996,49984,3955,48801
top,2006.15607,EPTCS,Lorenzo Iorio,Achieving the Secrecy Capacity of Wiretap Chan...,12 pages,"Dans Design, Automation and Test in Europe - D...",10.1021/jp806464p,ISSN 1947 5500,cs.CV,http://arxiv.org/licenses/nonexclusive-distrib...,This paper has been withdrawn.\n,"[{'version': 'v1', 'created': 'Wed, 27 Jun 201...",2007-05-23,"[['Iorio', 'Lorenzo', '']]"
freq,1,216,11,2,121,7,2,8,3105,36525,2,7,1016,11


In [40]:
# Calculate the percentage of null values in each column
null_percentages = df.isnull().sum() / len(df) * 100

# Print the results
print(null_percentages)

id                50.000
submitter         50.041
authors           50.000
title             50.000
comments          68.220
journal-ref       89.217
doi               84.635
report-no         98.830
categories        50.000
license           52.532
abstract          50.000
versions          50.000
update_date       50.000
authors_parsed    50.000
dtype: float64


In [41]:
# count the total number of missing values
df.isnull().values.sum()

843475

In [42]:

df.dropna(subset=["abstract"], inplace= True)


In [43]:

df["abstract"].isnull().sum()


0

### **Summarize your answers to Q1 below, based on your analysis:**


- We can know the features types from `info() method` 
    - The data type of this data in our dataset is  object in the total 14 columns.
- We can know the basic statistics from `describe(include='all')` method as shown in code.
- The number of features which having missing values = 14 columns as all the features have null values as shown in code.
  - I will handle the null values  only in abstract column.

Question 2: Preprocess the abstracts of the papers in the provided data corpus (note, in the tutorial, we use titles, not abstracts)
Apply bertopic on the abstracts and explain the topics you received and discuss the quality of the topics and determine what is the optimal topic number for abstracts.

In [44]:
# put your code for Q2 here, you can have multiple code blocks.
# Now let's create a tokenizer 
parser = en_core_sci_lg.load()
parser.max_length = 7000000 #Limit the size of the parser
punctuations = string.punctuation #list of punctuation to remove from text
stopwords = list(STOP_WORDS)


def spacy_tokenizer(sentence):
    ''' Function to preprocess text of scientific papers 
        (e.g Removing Stopword and puntuations)'''
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ] # transform to lowercase and then split the scentence
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ] #remove stopsword an punctuation
    mytokens = " ".join([i for i in mytokens]) 
    return mytokens

In [45]:
# Apply the tokenizer to process abstract, we will use the processed_text as the input for clustering task, but feel free to compare with just using the raw titles.
tqdm.pandas()

# Function to handle float values in the "abstract" column
def handle_float_value(value):
    if isinstance(value, float):
        return str(value)  # Convert float to string
    return value

# Apply the tokenizer to the "abstract" column
df["processed_text"] = df["abstract"].apply(handle_float_value).progress_apply(spacy_tokenizer)


100%|██████████| 50000/50000 [31:07<00:00, 26.77it/s]


In [46]:
df["processed_text"]

0        safety-critical system surgical robot autonomo...
1        entanglement key quantum resource improve meas...
2        collaborative editing consist edit common docu...
3        extend variational autoencoder vae collaborati...
4        slender structure immerse cross flow experienc...
                               ...                        
49995    analysis 1h-nmr spectrum hinder large variatio...
49996    thrust benefit lateral configuration two-dimen...
49997    constant magnetic field frequently discuss kno...
49998    virtual mental health assistants vmha prevalen...
49999    problem estimate frequency moment data stream ...
Name: processed_text, Length: 50000, dtype: object

In [47]:
df['abstract']

0          Since many safety-critical systems, such as ...
1          Entanglement is the key quantum resource for...
2          Collaborative editing consists in editing a ...
3          We extend variational autoencoders (VAEs) to...
4          Slender structures immersed in a cross flow ...
                               ...                        
49995      Analysis of 1H-NMR spectra is often hindered...
49996      The thrust benefits of lateral configuration...
49997      A constant magnetic field has frequently bee...
49998      Virtual Mental Health Assistants (VMHAs) hav...
49999      The problem of estimating frequency moments ...
Name: abstract, Length: 50000, dtype: object

In [48]:
# now let's create a topic model using BERTopic 

# since we have quite a lot data points, let's start with a large topic number (50) and see how coherent are the learned topics.
topic_model = BERTopic(verbose=True, embedding_model="paraphrase-MiniLM-L6-v2", min_topic_size=50)
topics, _ = topic_model.fit_transform(df["processed_text"].to_numpy()); len(topic_model.get_topic_info())

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

2023-06-06 00:01:47,533 - BERTopic - Transformed documents to Embeddings
2023-06-06 00:02:41,325 - BERTopic - Reduced dimensionality
2023-06-06 00:02:48,977 - BERTopic - Clustered reduced embeddings


100

### Topic Representation and visulization

In [49]:
# the following commands would help you to explore the topics generated. For the details of those functions, check bertopic functions.
topic_model.get_topic_info().head(10)
topic_model.visualize_barchart(top_n_topics=10, height=700)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,26323,-1_model_use_datum_method,"[model, use, datum, method, propose, result, s...",[simultaneous localization mapping slam algori...
1,0,1625,0_channel_interference_antenna_wireless,"[channel, interference, antenna, wireless, use...",[paper performance power domain downlink multi...
2,1,901,1_image_segmentation_imaging_medical,"[image, segmentation, imaging, medical, ct, mr...",[image segmentation important task medical app...
3,2,848,2_logic_program_calculus_proof,"[logic, program, calculus, proof, language, se...",[conference hold marseille 1982 iclp premier i...
4,3,806,3_optical_photonic_mode_waveguide,"[optical, photonic, mode, waveguide, cavity, l...",[report optical transmission measurement suspe...
5,4,722,4_equation_numerical_method_solution,"[equation, numerical, method, solution, scheme...",[work propose novel framework numerical soluti...
6,5,707,5_robot_robotic_human_control,"[robot, robotic, human, control, object, task,...",[establish common ground intelligent robot hum...
7,6,707,6_graph_vertex_edge_problem,"[graph, vertex, edge, problem, algorithm, log,...",[coloring graph assignment color vertex adjace...
8,7,683,7_neural_dnn_network_deep,"[neural, dnn, network, deep, layer, training, ...",[deep neural networks dnn computationally memo...
9,8,636,8_speech_speaker_audio_sound,"[speech, speaker, audio, sound, music, asr, ac...",[paper propose novel auxiliary loss function t...


In [50]:
topic_model.visualize_term_rank()

### Topic Relationships

In [51]:
topic_model.visualize_topics(top_n_topics=21)

In [52]:

topic_model.visualize_heatmap(n_clusters=5, top_n_topics=21)

In [53]:

topic_model.visualize_hierarchy(top_n_topics=21, width=800)

In [54]:
#reduce the number of topics to 90 if you see that's good to do it so it will reduce the number from 100 TO 90 topics 

topic_model.reduce_topics(df["processed_text"].to_numpy(),nr_topics=90)
topics = topic_model.topics_

2023-06-06 00:03:06,946 - BERTopic - Reduced number of topics from 100 to 90


In [55]:
#reduce the number of topics to 90 if you see that's good to do it

topic_model.reduce_topics(df["processed_text"].to_numpy(),nr_topics=20)
topics = topic_model.topics_

2023-06-06 00:03:17,076 - BERTopic - Reduced number of topics from 90 to 20


### **Summarize your answers to Q2 below, based on your analysis:**



- In this question, I preprocess the abstract of papers in our data as follow
  - first, create the tokenizer to process abstract
  -  Make Function to handle float values in the "abstract" column
  - Apply the tokenizer to process abstract
  - create a topic model using BERTopic with parameter min_topic_size=50
  - Apply different method on model to visulize it well as shown as follow in code.
  -  The number of Clustered reduced embeddings = 100 and that is big so I will fine model to reduce the number of clustered and I have got `90 clusters ` but this number is still big so i will reduce again and finally the number of clusters is `20 clusters` from this model.
- For the Topics I have recieved 
  - 26323	 topics is misclassified as they have `(-1)` but on the other hand the last of topics are classified well in topics' numbers which start from 0  as we show the head of topics from 0 to 9 topics.
  - I think the Optimal topic number for abstract will differ from one to other, for me i use the parameter  `min_topic_size=50` and it did well with me .

Question 3: Compare the topics learned by BERTopic with the original categories of the papers. 
For instance, you can pick one popular category, such as cs.CV, and check how those papers belong to this category are cateogrized by your topic model learned in previous step. 

In [31]:
# put your code for Q3 here, you can have multiple code blocks.
# add column topics to original dataset.
df['topics']=topics

In [32]:
new_df= df[df['categories']== 'cs.CV']

In [33]:
value_counts = new_df['topics'].value_counts().count()
value_counts

11

In [34]:
value_counts_ = new_df['topics'].value_counts()
value_counts_

-1     1736
 0     1184
 4       94
 2       47
 1       18
 15      14
 9        4
 8        4
 6        2
 14       1
 10       1
Name: topics, dtype: int64

In [35]:
# get the distribution of the dataframe
distribution = new_df.describe()
distribution

,topics
count,3105.000000
mean,-0.300805
std,1.529673
min,-1.000000
25%,-1.000000
50%,-1.000000
75%,0.000000
max,15.000000


### **Summarize your answers to Q3 below, based on your analysis:**


- When I Compare the topics learned by BERTopic with the original categories of the papers, I find the number of topics which is misclassified decrease as the number of topics which  misclassified  and have (-1) in the topic number is 1736 topics and the number of topics in general equal 11 for `cs.cv` category but that is not good enough as this is only one category so i will do redo and follow question 4.
- Also, the number of topic which was misclassified are still so big so i will retrain the model again.

Question 4: Redo the topic modeling using the same method, but only on the papers tagged by the category you chcked in Q3. For instance, in Q3, you pick cs.CV, then in Q4, you first filter data by category = cs.CV and then apply the topic modeling on the selected documents.
Compare your topics found and your results from Q3. 

In [29]:
# put your code for Q4 here, you can have multiple code blocks.

In [17]:
new_df=new_df.drop('topics', axis=1)

In [19]:
# since we have quite a lot data points, let's start with a large topic number (50) and see how coherent are the learned topics.
cs_model = BERTopic(verbose=True, embedding_model="paraphrase-MiniLM-L6-v2", min_topic_size=100)
topics_2, _ = topic_model.fit_transform(new_df["processed_text"].to_numpy()); len(topic_model.get_topic_info())

Batches:   0%|          | 0/98 [00:00<?, ?it/s]

2023-06-05 21:04:13,614 - BERTopic - Transformed documents to Embeddings
2023-06-05 21:04:32,820 - BERTopic - Reduced dimensionality
2023-06-05 21:04:33,517 - BERTopic - Clustered reduced embeddings


3

In [21]:
new_df['topic_2']=topics_2

In [22]:

value_counts_2 = new_df['topic_2'].value_counts()
value_counts_2

 0    2476
-1     421
 1     208
Name: topic_2, dtype: int64

### **Summarize your answers to Q4 below, based on your analysis:**

- Redo the topic modeling using the same method, but only on the papers tagged by the category you chcked in Q3. For instance, in Q3, you pick cs.CV, then in Q4, you first filter data by category = cs.CV and then apply the topic modeling on the selected documents. Compare your topics found and your results from Q3.
- First, I will drop the column topics from data and called it new_df dataframe - Then, retrain the model again with the preprocessed text and called it `cs_model`
- Then,save the result in topics_2 dataframe which I will add it to the new-df and calculate the counts of values .
- finally, I got `3 clusters` which is good enough number compare to previous tries also the number of topics which is misclassified is decrese alot and now equal `421 clusters` and that is a big progress here.